# NamedTuple
https://docs.python.org/3/library/collections.html#collections.namedtuple

As simple as tuple, but with better readability.

**Tuple**

In [20]:
x=1
y=2
point=(x,y)
point

(1, 2)

In [21]:
point[0]

1

In [22]:
point[1]

2

**NamedTuple**

In [23]:
from collections import namedtuple
Point=namedtuple("Point","x,y", defaults=[-1,-2])

x=1
y=2
point=Point(x,y)
point

Point(x=1, y=2)

In [24]:
point.x

1

In [25]:
point.y

2

In [26]:
point[0]

1

In [27]:
point[1]

2

In [28]:
point.y = 5

AttributeError: can't set attribute

In [ ]:
y[1] = 5

**Useful helper functions**  
(namedtuple methods start with single underscore. They are not private.)

In [29]:
point._replace(x=5)

Point(x=5, y=2)

In [30]:
point._asdict()

{'x': 1, 'y': 2}

In [31]:
point._fields

('x', 'y')

In [32]:
point._field_defaults

{'x': -1, 'y': -2}

In [33]:
Point()

Point(x=-1, y=-2)

# Dataclass
https://docs.python.org/3/library/dataclasses.html  
https://pyvideo.org/pycon-us-2018/dataclasses-the-code-generator-to-end-all-code-generators.html  
Not a replacement for namedtuple. Code generator to reduce common boilerplate.

Can be customized in many ways to do what we want.


In [35]:
from dataclasses import dataclass, field

@dataclass
class Point:
    x: int
    y: int

Point(1,2)


Point(x=1, y=2)

In [ ]:
@dataclass
class Point:
    x: int = -1
    y: int = -2

Point(y=6)

In [36]:
@dataclass
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6, repr=False) # Will not be in autogenerated repr

Point(1,2)

Point(x=1)

In [37]:
Point(1,2).y

2

In [44]:
@dataclass(repr=False) # Does not generate repr, so the dataclass inherits objects default repr
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2)

In [47]:
@dataclass(init=False) # __init__ method will not be generated
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2)

TypeError: Point() takes no arguments

In [46]:
point= Point()
point.x = 0
point

Point(x=0, y=-6)

**Dataclasses in sets**

In [50]:
@dataclass
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

{Point(1,2), Point(1,1)}

TypeError: unhashable type: 'Point'

In [53]:
point= Point(1,2)
point.x = 6

In [55]:
@dataclass(frozen=True)
class FrozenPoint:
    x: int = field(default=-1)
    y: int = field(default=-6)

{FrozenPoint(1,2), FrozenPoint(1,1)}


{FrozenPoint(x=1, y=1), FrozenPoint(x=1, y=2)}

In [ ]:
{FrozenPoint(1,1), FrozenPoint(1,1)}

In [54]:
point= FrozenPoint(1,2)
point.x = 6

FrozenInstanceError: cannot assign to field 'x'

In [56]:
@dataclass(unsafe_hash=True)
class HashablePoint:
    x: int = field(default=-1)
    y: int = field(default=-6)

{HashablePoint(1,1), HashablePoint(1,2)}

{HashablePoint(x=1, y=1), HashablePoint(x=1, y=2)}

In [66]:
@dataclass(unsafe_hash=True)
class HashablePoint:
    x: int = field(default=-1)
    y: int = field(default=-6, compare=False)

{HashablePoint(1,1), HashablePoint(1,2)}

{HashablePoint(x=1, y=1)}

In [67]:
point= HashablePoint(1,2)
point.x = 6

**Dataclasses in comparisons**

In [68]:
@dataclass()
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2) == Point(1,2)

True

In [72]:
@dataclass()
class Point:
    x: int = field(default=-1, compare=False)
    y: int = field(default=-6)

Point(666, 2) == Point(1,2)

True

In [70]:
Point(1,666) == Point(1,2)

False

In [73]:
{Point(666, 2), Point(1,2)}

TypeError: unhashable type: 'Point'

**Mutable default arguments**

In [75]:
class A():
    def __init__(self, a = []):
        self.a = a

a1=A()
a2=A()
print(a1.a, a2.a)

[] []


In [79]:
a1.a.append(5)
print(a1.a, a2.a)

[5, 5, 5, 5] [5, 5, 5, 5]


In [80]:
a2.a.append(6)
print(a1.a, a2.a)

[5, 5, 5, 5, 6] [5, 5, 5, 5, 6]


In [82]:
@dataclass()
class A():
    a: list = field(default_factory=list)
    b: list = field(default_factory=lambda: [1])

a1=A()
a2=A()
print(a1.a, a2.a)

[] []


In [83]:
a1.a.append(5)
print(a1.a, a2.a)

[5] []


In [85]:
a1.b.append(5)
print(a1.b, a2.b)


[1, 5] [1]


**Inheritance**

In [87]:
@dataclass()
class Base():
    a: int = 0
    b: int = 1

@dataclass()
class Child(Base):
    b: int = 666

Child()

Child(a=0, b=666)

**Inheritance and default values!**
https://medium.com/@aniscampos/python-dataclass-inheritance-finally-686eaf60fbb5

In [88]:
@dataclass()
class Base():
    a: int = 0
    b: int

TypeError: non-default argument 'b' follows default argument

In [93]:
@dataclass()
class Base():
    a: int
    b: int = 0

In [109]:
@dataclass()
class Child(Base):
    c: int
    d: int = 2

TypeError: non-default argument 'c' follows default argument

**(kw_only in Python 3.10)**

In [110]:
@dataclass(kw_only=True)
class Child(Base):
    c: int

Child(a=1,c=5)

Child(a=1, b=0, c=5)